## Data Collection

In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import  pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## Data Preprocessing

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

##laod the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

## Tokenize the text-creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

c:\Users\rites\Desktop\Predict Next Word\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rites\Desktop\Predict Next Word\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rites\Desktop\Predict Next Word\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. P

4818

## create input sequences

In [4]:
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)
      

## Pad Sequences

In [5]:
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [6]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]],
      shape=(25732, 14), dtype=int32)

## create predicitors and label

In [7]:
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

# Split the data into training and testing sets

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
# Define early stopping
#from tensorflow.keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## MODEL

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU,Bidirectional

## Define the model
model=Sequential()
model.add(Embedding(total_words, 100, input_shape=(max_sequence_len-1,)))

model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(100))
model.add(Dropout(0.3))

model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

c:\Users\rites\Desktop\Predict Next Word\venv\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [11]:
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)


Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.0320 - loss: 6.8352 - val_accuracy: 0.0344 - val_loss: 6.6806
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0364 - loss: 6.4874 - val_accuracy: 0.0381 - val_loss: 6.7535
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0411 - loss: 6.3829 - val_accuracy: 0.0418 - val_loss: 6.8301
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0487 - loss: 6.2788 - val_accuracy: 0.0519 - val_loss: 6.8619
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.0539 - loss: 6.1648 - val_accuracy: 0.0501 - val_loss: 6.8716
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.0577 - loss: 6.0521 - val_accuracy: 0.0521 - val_loss: 6.9126
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.0635 - loss: 5.9347 - val_accuracy: 0.0606 - val_loss: 7.0033
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.0736 - loss: 5.8152 - 

# Function to predict the next word

In [12]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [13]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction:{next_word}")

Input text:To be or not to be
Next Word Prediction:in't


## Save the model and tokenizer

In [14]:
## Save the model
model.save("next_word_lstm.h5")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
input_text="who is the"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction:{next_word}")

Input text:who is the
Next Word Prediction:king
